In [ ]:
import matplotlib.pyplot as plt

from analysis_function import *

import json
import pickle

In [ ]:
prompt_file = "prompt_file"
generation_file = "generation_file"

In [ ]:
#open json file "prompts.json" and load the data
with open(prompt_file, "r") as f:
    prompts = json.load(f)

#open file using pickle
with open(generation_file, "rb") as file:
    data = pickle.load(file)

In [ ]:
sample_per_prompt = 75
filtering = []

topic = []
total_user = []
difference = []

prompts["prompts"] = prompts["prompts"][:len(data)//sample_per_prompt]

for prompt in prompts["prompts"]:
    item = {"topic": prompt["topic"],
            "total_user": prompt["total_user"],
            "difference": prompt["difference"]
            }
    
    filtering = filtering + [item] * sample_per_prompt

In [ ]:
data_relaxed = []

for conv in data:
    try:
        test = json.loads(conv)

        test_reman = remanaged_conversation(test)

        data_relaxed.append(json.dumps(test_reman))
    except:
        data_relaxed.append(conv)

In [ ]:
good_format, data_good, filtering_good, good_speakers, good_turns, good_first_addressees, good_addr, good_stances, good_everyone_speaks = full_checks(data_relaxed, filtering)

In [ ]:
remanaged_data = []

for conv in data_good:
    test = json.loads(conv)
        
    if len(test["conversation"])>0:
        test["conversation"][0]["addressee"] = []
        for sp in test["speakers"]:
            if sp["name"] != test["conversation"][0]["speaker"]:
                test["conversation"][0]["addressee"].append(sp["name"])  
    
    remanaged_data.append(json.dumps(test))


good_addr_rem = check_number_of_first_addr(remanaged_data)

# Determine the bins based on the combined data range
bins = np.linspace(min(good_addr_rem), max(good_addr_rem), 3)  # Adjust number of bins as needed

# Create subplots

# Plot each histogram

plt.hist(good_addr_rem, bins=bins, density=True, color='lightgreen', edgecolor='black')

# Adjust layout
plt.tight_layout()
plt.show()

good_addr = check_no_wrong_addressees(remanaged_data)

# Determine the bins based on the combined data range
bins = np.linspace(min(good_addr), max(good_addr), 3)  # Adjust number of bins as needed

# Create subplots

# Plot each histogram

plt.hist(good_addr, bins=bins, density=True, color='lightgreen', edgecolor='black')

# Adjust layout
plt.tight_layout()
plt.show()


good_more_avg2 = check_turns_greater_than_2(remanaged_data)

# Determine the bins based on the combined data range
bins = np.linspace(min(good_addr_rem), max(good_addr_rem), 3)  # Adjust number of bins as needed

# Create subplots

# Plot each histogram

plt.hist(good_more_avg2, bins=bins, density=True, color='lightgreen', edgecolor='black')

# Adjust layout
plt.tight_layout()
plt.show()
        


In [ ]:

final_selection = [x*y for x, y in zip (good_speakers,  good_addr_rem)]
final_selection = [x*y for x, y in zip (final_selection,  good_more_avg2)]
final_selection = [x*y for x, y in zip (final_selection,  good_stances)]
final_selection = [x*y for x, y in zip (final_selection,  good_addr)]
final_selection = [x*y for x, y in zip (final_selection,  good_everyone_speaks)]




print(sum(final_selection))
print(len(final_selection))

# Determine the bins based on the combined data range
bins = np.linspace(min(final_selection), max(final_selection), 3)  # Adjust number of bins as needed


plt.hist(final_selection, bins=bins, density=True, color='lightgreen', edgecolor='black')

# Adjust layout
plt.tight_layout()
plt.show()

final_good = [x for x, y in zip(remanaged_data, final_selection) if y]
final_filtering_good = [x for x, y in zip(filtering_good, final_selection) if y]

In [ ]:
#save the final selection

final_good_file = "final_good_file"
final_filtering_good_file = "final_filtering_good_file"

with open(final_good_file, 'wb') as file:
    pickle.dump(final_good, file)
    
with open(final_filtering_good_file, 'wb') as file:
    pickle.dump(final_filtering_good, file)

In [ ]:
#average number of addressees per turn
avg_addressee = []
number_speakers = []

number_of_turns = []

categories = []

for data in final_good:
    test = json.loads(data)
    
    addressees = []
    for turn in test["conversation"]:
        addressees.append(len(turn["addressee"])/(len(test["speakers"])-1))
        
    avg_addressee.append(sum(addressees)/len(addressees))
    number_speakers.append(len(test["speakers"]))
    number_of_turns.append(len(test["conversation"]))
    pos = 0
    neg = 0 
    for stance in test["speakers"]:
        if stance["stance"] == "positive":
            pos += 1
        elif stance["stance"] == "negative":
            neg += 1
    categories.append(str(pos) + "-" + str(neg))
    

import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde


data_continuous = np.array(avg_addressee)
data_continuous[np.isnan(data_continuous)] = 0

fig, ax = plt.subplots()
ax.ecdf(data_continuous, color='blue', label='ECDF all', alpha=1)
# Plot the PDF
ax.set_xlim(0, 1)

colors = ["green", "red", "purple"]

conv_speak = [[],[],[]]

conv_speak[0] = [i for i, g in enumerate(number_speakers) if g == 4]
conv_speak[1] = [i for i, g in enumerate(number_speakers) if g == 5]
conv_speak[2] = [i for i, g in enumerate(number_speakers) if g == 6]

for j in range(3):
    metric_sel = [avg_addressee[i] for i in conv_speak[j]]

    #convert metric in a numpy array
    data_continuous = np.array(metric_sel)
    data_continuous[np.isnan(data_continuous)] = 0
    # Kernel Density Estimation (KDE)
    ax.ecdf(data_continuous, color=colors[j], label="ECDF " + str(j+4) + " nodes", linestyle='dashed', alpha=0.75, linewidth=0.75)

plt.title('Empirical Cumulative Density Function (ECDF)')
plt.xlabel("average #addressee per turn")
plt.ylabel('Density')
plt.legend()
plt.grid(alpha=0.4)

svg_filename = ("addr_per_turn.svg")
plt.savefig(svg_filename, format='svg')
plt.close()  # Close the plot to release memory

import numpy as np
labels, counts = np.unique(number_speakers, return_counts=True)

ticks = range(len(counts))
plt.bar(ticks,counts, align='center')
plt.xticks(ticks, labels)
# Adjust layout
plt.tight_layout()

svg_filename = ("number_users.svg")
plt.savefig(svg_filename, format='svg')
plt.close()  # Close the plot to release memory


import numpy as np
labels, counts = np.unique(number_of_turns,return_counts=True)

ticks = range(len(counts))
plt.bar(ticks,counts, align='center')
plt.xticks(ticks, labels)
# Adjust layout
plt.tight_layout()

svg_filename = ("conversation_length.svg")
plt.savefig(svg_filename, format='svg')
plt.close()  # Close the plot to release memory


import numpy as np
labels, counts = np.unique(categories,return_counts=True)

ticks = range(len(counts))
plt.bar(ticks,counts, align='center')
plt.xticks(ticks, labels)
# Adjust layout
plt.tight_layout()

svg_filename = ("stance_balance.svg")
plt.savefig(svg_filename, format='svg')
plt.close()  # Close the plot to release memory